# Web and Text Analytics - Sentiment Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Experiment 1: Hyperparameters
In this experiment, we compare RNN, LSTM and GRU over the IMDB datasets and examine the results by varying hyperparameters.

List hparams and explain.

## Experiment 2: Attention
In this experiment, we analyse the effect of adding an attention layer for the sentiment analysis.